In [5]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

 ### Parameter Test
inputType ='influx' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
imputation_param ={
"imputation_method":[
    {"min":0,"max":1,"method":"mean"},
    {"min":2,"max":4,"method":"linear"},
    {"min":5,"max":10,"method":"brits"}],
"totalNanLimit":0.3}

import makeNaNImputationTest as IT
input_data = IT.inputControl(inputType)

from KETIPrePartialDataPreprocessing import data_preprocessing
output = data_preprocessing.get_preprocessed_data(input_data, refine_param, outlier_param, imputation_param)

original in_ciai          0
in_cici          0
in_cici_co2      0
in_cici_humi     0
in_cici_noise    0
in_cici_pm10     0
in_cici_pm25     0
in_cici_temp     0
in_cici_voc      0
in_co2           0
in_humi          0
in_noise         0
in_pm01          0
in_pm01_raw      0
in_pm10          0
in_pm10_raw      0
in_pm25          0
in_pm25_raw      0
in_temp          0
in_voc           0
dtype: int64
after refine in_ciai          5686
in_cici          5686
in_cici_co2      5686
in_cici_humi     5686
in_cici_noise    5686
in_cici_pm10     5686
in_cici_pm25     5686
in_cici_temp     5686
in_cici_voc      5686
in_co2           5686
in_humi          5686
in_noise         5686
in_pm01          5686
in_pm01_raw      5686
in_pm10          5686
in_pm10_raw      5686
in_pm25          5686
in_pm25_raw      5686
in_temp          5686
in_voc           5686
dtype: int64
after outlierDetection in_ciai          5686
in_cici          5686
in_cici_co2      5777
in_cici_humi     5777
in_cici_noise    5777